# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 64 new papers today
          26 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/25 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.10778


extracting tarball to tmp_2210.10778...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.10789


extracting tarball to tmp_2210.10789...

...

extracting tarball to tmp_2210.11360...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.11424


extracting tarball to tmp_2210.11424...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.11424/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'Appendix' from 'tmp_2210.11424/Appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'Discussion' from 'tmp_2210.11424/Discussion.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_dec

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11103-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11103) | **Lower-than-expected flare temperatures for TRAPPIST-1**  |
|| A. J. Maas, et al. -- incl., <mark>C.-L. Lin</mark> |
|*Appeared on*| *2022-10-21*|
|*Comments*| *17 pages, 11 figures, 2 tables, accepted for publication in A&A*|
|**Abstract**| Although high energetic radiation from flares is a potential threat to exoplanet atmospheres and may lead to surface sterilization, it might also provide the extra energy for low-mass stars needed to trigger and sustain prebiotic chemistry. We investigate two flares on TRAPPIST-1, an ultra-cool dwarf star that hosts seven exoplanets of which three lie within its habitable zone. The flares are detected in all four passbands of the MuSCAT2 allowing a determination of their temperatures and bolometric energies. We analyzed the light curves of the MuSCAT1 and MuSCAT2 instruments obtained between 2016 and 2021 in $g,r,i,z_\mathrm{s}$-filters. We conducted an automated flare search and visually confirmed possible flare events. We studied the temperature evolution, the global temperature, and the peak temperature of both flares. For the first time we infer effective black body temperatures of flares that occurred on TRAPPIST-1. The black body temperatures for the two TRAPPIST-1 flares derived from the SED are consistent with $T_\mathrm{SED} = 7940_{-390}^{+430}$K and $T_\mathrm{SED} = 6030_{-270}^{+300}$K. The flare black body temperatures at the peak are also calculated from the peak SED yielding $T_\mathrm{SEDp} = 13620_{-1220}^{1520}$K and $T_\mathrm{SEDp} = 8290_{-550}^{+660}$K. We show that for the ultra-cool M-dwarf TRAPPIST-1 the flare black body temperatures associated with the total continuum emission are lower and not consistent with the usually adopted assumption of 9000-10000 K. This could imply different and faster cooling mechanisms. Further multi-color observations are needed to investigate whether or not our observations are a general characteristic of ultra-cool M-dwarfs. This would have significant implications for the habitability of exoplanets around these stars because the UV surface flux is likely to be overestimated by the models with higher flare temperatures. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.10909-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.10909) | **TOI-3884 b: A rare 6-R$_{\oplus}$ planet that transits a low-mass star  with a giant and likely polar spot**  |
|| J.M. Almenara, et al. -- incl., <mark>R.P. Schwarz</mark>, <mark>J.J. Lissauer</mark> |
|*Appeared on*| *2022-10-21*|
|*Comments*| *13 pages, 13 figures, accepted for publication in A&A Letters*|
|**Abstract**| The Transiting Exoplanet Survey Satellite mission identified a deep and asymmetric transit-like signal with a periodicity of 4.5 days orbiting the M4 dwarf star TOI-3884. The signal has been confirmed by follow-up observations collected by the ExTrA facility and Las Cumbres Observatory Global Telescope, which reveal that the transit is chromatic. The light curves are well modelled by a host star having a large polar spot transited by a 6-R$_{\oplus}$ planet. We validate the planet with seeing-limited photometry, high-resolution imaging, and radial velocities. TOI-3884 b, with a radius of $6.00 \pm 0.18$ R$_{\oplus}$, is the first sub-Saturn planet transiting a mid-M dwarf. Owing to the host star's brightness and small size, it has one of the largest transmission spectroscopy metrics for this planet size and becomes a top target for atmospheric characterisation with the James Webb Space Telescope and ground-based telescopes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.10778-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.10778) | **Standard physics is still capable to interpret $\sim$18 TeV photons from  GRB~221009A**  |
|| <mark>Zhi-Chao Zhao</mark>, Yong Zhou, Sai Wang |
|*Appeared on*| *2022-10-21*|
|*Comments*| *4 pages, 2 figures*|
|**Abstract**| It is reported that the Large High Altitude Air Shower Observatory (LHAASO) observed thousands of very-high-energy photons up to $\sim$18 TeV from GRB 221009A. We study the survival rate of these photons by considering the fact that they are absorbed by the extragalactic background light. By performing a set of $10^6$ Monte-Carlo simulations, we explore the parameter space allowed by current observations and find that the probability of predicting that LHAASO observes at least one photons of 18 TeV from GRB 221009A within 2000 seconds is 4--5\%. Hence, it is still possible for the standard physics to interpret LHAASO's observation in the energy range of several TeV. Our method can be straightforwardly generalized to study more data sets of LHAASO and other experiments in the future. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11293-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11293) | **The next generation Birmingham Solar Oscillations Network (BiSON)  spectrophotometer: a new miniaturised instrument for helioseismology**  |
|| S. J. Hale, et al. -- incl., <mark>G. R. Davies</mark> |
|*Appeared on*| *2022-10-21*|
|*Comments*| *8 pages, 8 figures, 1 table. Accepted by RAS Techniques and Instruments: 2022 September 26*|
|**Abstract**| We describe a new spectrophotometer for the Birmingham Solar Oscillations Network (BiSON), based on a next generation observation platform, BiSON:NG, a significantly miniaturised system making use of inexpensive consumer-grade hardware and off-the-shelf components, where possible. We show through system modelling and simulation, along with a summer observing campaign, that the prototype instrument produces data on the Sun's low-degree acoustic (p-mode) oscillations that are of equal quality and can be seamlessly integrated into the existing network. Refreshing the existing ageing hardware, and the extended observational network potential of BiSON:NG, will secure our ongoing programme of high-quality synoptic observations of the Sun's low-degree oscillations (e.g., for seismic monitoring of the solar cycle at a "whole Sun" level). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11329-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11329) | **Estimating the Convective Turnover Time**  |
|| Wei-Chun Jao, et al. -- incl., <mark>Todd J. Henry</mark> |
|*Appeared on*| *2022-10-21*|
|*Comments*| *8 pages, 2 figures, 1 table. One of the figures is a 3D interactive plot, which is available in the online journal or at this https URL*|
|**Abstract**| The introduction of the Rossby number (R$_0$), which incorporates the convective turnover time ($\tau$), in 1984 was a pioneering idea for understanding the correlation between stellar rotation and activity. The convective turnover time, which cannot be measured directly, is often inferred using existing $\tau$-mass or $\tau$-color relations, typically established based on an ensemble of different types of stars by assuming that $\tau$ is a function of mass. In this work, we use {\it Gaia} Early Data Release 3 to demonstrate that the masses used to establish one of the most cited $\tau$-mass relations are overestimated for G type dwarfs and significantly underestimated for late M dwarfs, offsets that affect studies using this $\tau$-mass relation to draw conclusions. We discuss the challenges of creating such relations then and now. In the era of {\it Gaia} and other large datasets, stars used to establish these relations require characterization in a multi-dimensional space, rather than via the single-characteristic relations of the past. We propose that new multi-dimensional relations should be established based on updated theoretical models and all available stellar parameters for different interior structures from a set of carefully vetted single stars, so that the convective turnover time can be estimated more accurately. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11333-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11333) | **The multipole expansion of the local expansion rate**  |
|| <mark>Basheer Kalbouneh</mark>, Christian Marinoni, Julien Bel |
|*Appeared on*| *2022-10-21*|
|*Comments*| *26 pages, 16 figures*|
|**Abstract**| We design a new observable, the expansion rate fluctuation $\eta$, to characterize deviations from the linear relation between redshift and distance in the local universe. We also show how to compress the resulting signal into spherical harmonic coefficients in order to better decipher the structure and symmetries of the anisotropies in the local expansion rate. We apply this analysis scheme to several public catalogs of redshift-independent distances, the Cosmicflows-3 and Pantheon data sets, covering the redshift range $0.01<z<0.05$. The leading anisotropic signal is stored in the dipole. Within the standard cosmological model, it is interpreted as a bulk motion ($307 \pm 23$ km/s) of the entire local volume in a direction aligned at better than $4$ degrees with the bulk component of the Local Group velocity with respect to the CMB. This term alone, however, provides an overly simplistic and inaccurate description of the angular anisotropies of the expansion rate. We find that the quadrupole contribution is non-negligible ($\sim 50\%$ of the anisotropic signal), in fact, statistically significant, and signaling a substantial shearing of gravity in the volume covered by the data. In addition, the 3D structure of the quadrupole is axisymmetric, with the expansion axis aligned along the axis of the dipole. Implications for the determination of the $H_0$ parameter are also discussed. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11360-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11360) | **An X-ray Dim Isolated Neutron Star in Binary?**  |
|| <mark>Jie Lin</mark>, et al. -- incl., <mark>Chunqian Li</mark>, <mark>Heng Xu</mark> |
|*Appeared on*| *2022-10-21*|
|*Comments*| *15 pages, 9 figures*|
|**Abstract**| We report the discovery of a dark companion to 2MASS J15274848+3536572 with an orbital period of 6.14 hours. Combining the radial velocity of LAMOST observation and the modelling of the multi-band light curve, one has a mass function of 0.135 Msun, an inclination of 43.94+0.33-0.21, and a mass ratio of 0.58+0.048-0.018, which demonstrate the binary nature of a dark companion with mass of 1.01+-0.08Msun and a main-sequence K star of 0.59+-0.05 Msun. LAMOST optical spectra at a range of orbital phase reveals extra peaked Halpha emission that suggests the presence of an accretion disk. The dark companion does not seem to be a white dwarf because the lack of any observed dwarf nova outbursts contradicts with the disk instability model in long-term data archive. Alternatively, we propose a scenario that the dark companion is a neutron star, but we have not detected radio pulsation or single pulse from the system with the FAST (Five hundred meter Aperture Spherical radio Telescope), which hints a radio quiet compact object. If the dark companion is identified as a neutron star, it will be nearest (118 pc) and lightest neutron star. Furthermore, a kinematic analysis of the systems orbit in the galaxy may suggest its supernova event is associated with the radionuclide 60Fe signal observed from the deep-sea crusts. This radio-quiet and X-ray dim nearby neutron star may resemble an XDINS (X-ray dim isolated neutron star), but in a binary. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11424-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11424) | **SDSS IV MaNGA: Bar pattern speed in Milky Way Analogue galaxies**  |
|| Luis Garma-Oehmichen, et al. -- incl., <mark>Héctor Hernández-Toledo</mark>, <mark>José Antonio Vázquez-Mata</mark>, <mark>Luis Artemio Martínez-Vázquez</mark> |
|*Appeared on*| *2022-10-21*|
|*Comments*| *19 pages, 20 figures, 2 tables, accepted for publication in MNRAS*|
|**Abstract**| Most secular effects produced by stellar bars strongly depend on the pattern speed. Unfortunately, it is also the most difficult observational parameter to estimate. In this work, we measured the bar pattern speed of 97 Milky-Way Analogue galaxies from the MaNGA survey using the Tremaine-Weinberg method. The sample was selected by constraining the stellar mass and morphological type. We improve our measurements by weighting three independent estimates of the disc position angle. To recover the disc rotation curve, we fit a kinematic model to the H$_\alpha$ velocity maps correcting for the non-circular motions produced by the bar. The complete sample has a smooth distribution of the bar pattern speed ($\Omega_{Bar}=28.14^{+12.30}_{-9.55}$ km s$^{-1}$ kpc $^{-1}$), corotation radius ($R_{CR} = 7.82^{+3.99}_{-2.96}$ kpc) and the rotation rate ($\mathcal{R} = 1.35^{+0.60}_{-0.40}$). We found two sets of correlations: (i) between the bar pattern speed, the bar length and the logarithmic stellar mass (ii) between the bar pattern speed, the disc circular velocity and the bar rotation rate. If we constrain our sample by inclination within $30 \degree < i < 60 \degree$ and relative orientation $20\degree<|PA_{disc}-PA_{bar} |<70\degree$, the correlations become stronger and the fraction of ultra-fast bars is reduced from 20\% to 10\% of the sample. This suggest that a significant fraction of ultra-fast bars in our sample could be associated to the geometric limitations of the TW-method. By further constraining the bar size and disc circular velocity, we obtain a sub-sample of 25 Milky-Way analogues galaxies with distributions $\Omega_{Bar}=30.48^{+10.94}_{-6.57}$ km s$^{-1}$ kpc$^{-1}$, $R_{CR} = 6.77^{+2.32}_{-1.91}$ kpc and $\mathcal{R} = 1.45^{+0.57}_{-0.43}$, in good agreement with the current estimations for our Galaxy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.11103.md
    + _build/html/tmp_2210.11103/./figures/paper_figure_9_1.png
    + _build/html/tmp_2210.11103/./figures/paper_figure_1.png
    + _build/html/tmp_2210.11103/./figures/paper_figure_8.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# Lower-than-expected flare temperatures for TRAPPIST-1

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.11103-b31b1b.svg)](https://arxiv.org/abs/2210.11103)<mark>Appeared on: 2022-10-21</mark> - _17 pages, 11 figures, 2 tables, accepted for publication in A&A_

</div>
<div id="authors">

A. J. Maas, et al. -- incl., <mark>C.-L. Lin</mark>

</div>
<div id="abstract">

**Abstract:** Although high energetic radiation from flares is a potential threat to exoplanet atmospheres and may lead to surface sterilization, it might also provide the extra energy for low-mass stars needed to trigger and sustain prebiotic chemistry. We investigate two flares on TRAPPIST-1, an ultra-cool dwarf star that hosts seven exoplanets of which three lie within its habitable zone. The flares are detected in all four passbands of the MuSCAT2 allowing a determination of their temperatures and bolometric energies. We analyzed the light curves of the MuSCAT1 and MuSCAT2 instruments obtained between 2016 and 2021 in $g,r,i,z_\mathrm{s}$-filters. We conducted an automated flare search and visually confirmed possible flare events. We studied the temperature evolution, the global temperature, and the peak temperature of both flares. For the first time we infer effective black body temperatures of flares that occurred on TRAPPIST-1. The black body temperatures for the two TRAPPIST-1 flares derived from the SED are consistent with $T_\mathrm{SED} = 7940_{-390}^{+430}$K and $T_\mathrm{SED} = 6030_{-270}^{+300}$K. The flare black body temperatures at the peak are also calculated from the peak SED yielding $T_\mathrm{SEDp} = 13620_{-1220}^{1520}$K and $T_\mathrm{SEDp} = 8290_{-550}^{+660}$K. We show that for the ultra-cool M-dwarf TRAPPIST-1 the flare black body temperatures associated with the total continuum emission are lower and not consistent with the usually adopted assumption of 9000-10000 K. This could imply different and faster cooling mechanisms. Further multi-color observations are needed to investigate whether or not our observations are a general characteristic of ultra-cool M-dwarfs. This would have significant implications for the habitability of exoplanets around these stars because the UV surface flux is likely to be overestimated by the models with higher flare temperatures. 

</div>

<div id="div_fig1">

<img src="tmp_2210.11103/./figures/paper_figure_9_1.png" alt="Fig8" width="100%"/>

**Figure 8. -** Spectral type and stellar effective temperature versus total flare temperature for different photometric and spectroscopic studies: The upper panel uses the stellar effective temperature if available from the TESS input catalogue \citep{Stassun_2019}. For the Kowalski sample, we use conservative uncertainties of $\pm 500$K. The lower panel shows a density map of flare temperatures per spectral type that is consistent with the upper stellar effective temperatures. The flare temperature axis is represented in log-scale. The two TRAPPIST-1 flares are overplotted and the blue horizontal lines represent the average temperatures from the two biggest temperature samples. (*fig:fig9*)

</div>
<div id="div_fig2">

<img src="tmp_2210.11103/./figures/paper_figure_1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Light curve in all four filters of MuSCAT2 (M2) of Flare 1: The flare is visible in the $g$, $r$, and $i$ filter, whereas the S/N in the $z_\mathrm{s}$-filter is too low. To better distinguish between the four M2 channels, we add constant values to the flux. The time is given in days in reduced Julian date (RJD), RJD = BJD - 2454833 d where BJD is the barycentric Julian date.  (*fig:observation*)

</div>
<div id="div_fig3">

<img src="tmp_2210.11103/./figures/paper_figure_8.png" alt="Fig11" width="100%"/>

**Figure 11. -** Flare frequency distribution for K2 and M2 data. The FFD for TRAPPIST-1 is updated by our flares indicated in orange. The black points are adopted from \citep{Paudel_2018}. The best fit to the power law is chosen from 2500 MC trials, here only 100 random samples are shown to mark the uncertainty of the model. Also overplotted are the abiogenesis zone in green \citep{Rimmer_2018,Gunther_2020} and the ozone-depletion zone in yellow \citep{2019_Tilley, Gunther_2020} for TRAPPIST-1. (*fig:FFD*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

70  publications in the last 7 days.
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers